# Limitations of reference-counted Rc and Arc types in Rust

The important property of `std::rc::Rc` is that clones of it can 
be moved around. Ownership of those values can be moved around 
your program, which can sometimes help to around tricky design
problems. However, it doesn't solve all problems.

Consider the code below. Note that we *move* each clone of the
`Rc` into the function being called:

In [40]:
use std::rc::Rc;

fn a(x: Rc<String>) {
    println!("{}", x);
}

fn b(y: Rc<String>) {
    println!("{}", y);
}

pub fn hey() {
    let s = "hey".to_string();
    let r = Rc::new(s);
    a(r.clone());
    b(r.clone());
}

hey()

hey
hey


()

Well the question is: what can you do with that? As far as mutability is 
concerned, not much.

The code below is almost the same, but here I'm trying to obtain a
mutable ref to the `String` inside one of the clones of the `Rc`:

In [18]:
fn a(mut x: Rc<String>) {
    let mut z = &*Rc::get_mut(&mut x).unwrap();
    println!("{}", z);
}

fn b(y: Rc<String>) {
    println!("{}", y);
}

pub fn hey() {
    let s = "hey".to_string();
    let r = Rc::new(s);
    a(r.clone());
    b(r.clone());
}

hey()

thread '<unnamed>' panicked at 'called `Option::unwrap()` on a `None` value', src/lib.rs:15:39
stack backtrace:
   0: rust_begin_unwind
             at /rustc/9bc8c42bb2f19e745a63f3445f1ac248fb015e53/library/std/src/panicking.rs:493:5
   1: core::panicking::panic_fmt
             at /rustc/9bc8c42bb2f19e745a63f3445f1ac248fb015e53/library/core/src/panicking.rs:92:14
   2: core::panicking::panic
             at /rustc/9bc8c42bb2f19e745a63f3445f1ac248fb015e53/library/core/src/panicking.rs:50:5
   3: run_user_code_5
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.
Segmentation fault.
   0: evcxr::runtime::Runtime::install_crash_handlers::segfault_handler
   1: <unknown>
   2: mi_free
   3: alloc::alloc::dealloc
             at /rustc/9bc8c42bb2f19e745a63f3445f1ac248fb015e53/library/alloc/src/alloc.rs:104:14
      <alloc::alloc::Global as core::alloc::A

Error: Child process terminated with status: signal: 6 (core dumped)

It doesn't go well. The problem here is that the `Option` you get back from `Rc::get_mut` returns `None` because
you're not allowed to mutate the string since it's being referenced in another `Rc` somewhere else (in function `b()`). So even though `Rc` does easily let you move ownership around (of the wrapper), it doesn't make mutation any easier.

However, `Rc` does let you do interesting things for read-only situations that would otherwise be cumbersome. For example, we can make two structs reference each other quite easily:

In [39]:
#[derive(Debug)]
struct Node {
    id: usize,
    other: Option<Rc<Node>>
}

let mut n1 = Node { id: 1, other: None };
let n2 = Node { id: 2, other: None };

n1.other = Some(Rc::new(n2));

let mut r = Rc::new(n1);
let mut n2 = &mut *Rc::get_mut(&mut r).unwrap();

//let mut n2 = &mut *Rc::get_mut(&mut n2r).unwrap();
//drop(n2);



fn attach(mut n: &mut Rc<Node>, other: &Rc<Node>) {
    let mut inner = &mut *Rc::get_mut(n).unwrap();
    inner.other = Some(other.clone());
}


(&r, 1)

(Node { id: 1, other: Some(Node { id: 2, other: None }) }, 1)